# Day 4 - Giant Squid

https://adventofcode.com/2021/day/4

## Part 1

Here I decided to go a little nuts with some OOP design.

I'll write out some helper methods first for parsing the inputs...

In [32]:
from typing import Generator
from pathlib import Path

INPUTS = Path('input.txt').read_text().strip().split('\n')

def numbers_drawn() -> Generator[int, None, None]:
    yield from (int(x) for x in INPUTS[0].split(','))

def bingo_board_inputs() -> Generator[list[list[int]], None, None]:
    board_inputs = INPUTS[2:]
    for i in range(0, len(board_inputs)-5, 6):
        # step by 6 skips the empty rows
        this_board = board_inputs[i:i+5]
        yield [[int(y) for y in x.split()] for x in this_board]

Next a `BingoBoard` class intended to store the relevant contents of a single board, calculate its win conditions, and produce its score.

For the win condition, I figured I can parse out the sets of numbers that would equal a win for the given board, by actually generating the number sets for their rows, columns, and diagonals. This way, we can more easily discover if a given board has won using set arithmetic: checking the union between every number drawn and the individual conditions. If the resulting union matches the full win condition, then the board has won!

Finally, the score is obtained through a similar means: subtracting the set of all marked numbers from the full set of numbers present in the board, we get a list of those remaining, unmarked numbers. A simple `sum()` call multiplied by the last drawn number (`marks[-1]` for last position of the list of marked numbers), and we get our score.

In [33]:
class BingoBoard:
    def __init__(self, board: list[list[int]]):
        self.original: list[list[int]] = board
        self.all_nums = set(x for y in board for x in y)
        self.win_conditions: list[set] = []
        self.marks: list = []
        self.calc_win_conditions()
    
    def __str__(self) -> str:
        return "\n".join(
            [
                " ".join([f"{y:>2}" for y in x])
                for x in self.original
            ]
        )

    def calc_win_conditions(self) -> None:
        # Set up the list of win condition sets of numbers from the input board
        self.win_conditions = []
        # Start with horizontals
        self.win_conditions.extend([set(x) for x in self.original])

        # Then verticals (transposed)
        transposed = list(map(list, zip(*self.original)))
        self.win_conditions.extend([set(x) for x in transposed])
        # Finally, the two diagonals
        self.win_conditions.append(
            {
                self.original[0][0],
                self.original[1][1],
                self.original[2][2],
                self.original[3][3],
                self.original[4][4],
            }
        )
        self.win_conditions.append(
            {
                self.original[0][4],
                self.original[1][3],
                self.original[2][2],
                self.original[3][1],
                self.original[4][0],
            }
        )

    def mark(self, num: int) -> None:
        self.marks.append(num)

    def is_winner(self) -> bool:
        return any((set(self.marks) & x == x) for x in self.win_conditions)

    @property
    def score(self) -> int:
        if not self.is_winner():
            return 0
        remaining = list(self.all_nums - (self.all_nums & set(self.marks)))
        return sum(remaining) * self.marks[-1]


def test_win_conditions() -> None:
    thing = BingoBoard(
        board=[
            [78, 13, 8, 62, 67],
            [42, 89, 97, 16, 65],
            [5, 12, 73, 50, 56],
            [45, 10, 63, 41, 64],
            [49, 1, 95, 71, 17],
        ]
    )
    expected = [
        {67, 8, 13, 78, 62},
        {97, 65, 42, 16, 89},
        {5, 73, 12, 50, 56},
        {64, 41, 10, 45, 63},
        {1, 71, 49, 17, 95},
        {5, 42, 45, 78, 49},
        {1, 10, 12, 13, 89},
        {97, 8, 73, 63, 95},
        {71, 41, 16, 50, 62},
        {64, 65, 67, 17, 56},
        {73, 41, 78, 17, 89},
        {67, 73, 10, 16, 49},
    ]
    assert thing.win_conditions == expected


def test_is_winner() -> None:
    thing = BingoBoard(
        board=[
            [78, 13, 8, 62, 67],
            [42, 89, 97, 16, 65],
            [5, 12, 73, 50, 56],
            [45, 10, 63, 41, 64],
            [49, 1, 95, 71, 17],
        ]
    )
    thing.mark(78)
    assert not thing.is_winner()

    thing.mark(89)
    thing.mark(8)
    thing.mark(62)
    assert not thing.is_winner()

    thing.mark(13)
    assert not thing.is_winner()

    thing.mark(67)
    assert thing.is_winner()


def test_scoring() -> None:
    thing = BingoBoard(
        board=[
            [14, 21, 17, 24, 4],
            [10, 16, 15, 9, 19],
            [18, 8, 23, 26, 20],
            [22, 11, 13, 6, 5],
            [2, 0, 12, 3, 7],
        ]
    )
    thing.marks = [7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24]
    assert thing.score == 4512


def run_tests() -> None:
    test_win_conditions()
    test_is_winner()
    test_scoring()


run_tests()


The final piece was a helper class to contain all boards, the `BoardSet`. This would iterate through every board, mark new numbers, check for winners, and be able to return that winning board.

In [35]:
class BoardSet:
    def __init__(self):
        self.boards: list[BingoBoard] = []

    def mark(self, num: int) -> None:
        for board in self.boards:
            board.mark(num=num)
    
    def has_winner(self) -> bool:
        return any([x.is_winner() for x in self.boards])
    
    def winning_board(self) -> BingoBoard:
        if self.has_winner():
            return [x for x in self.boards if x.is_winner()][0]


Now we can finally run the game and see who wins first.

In [37]:
board_set = BoardSet()
board_set.boards.extend(
    [BingoBoard(board=x) for x in bingo_board_inputs()]
)

for num in numbers_drawn():
    board_set.mark(num)
    if board_set.has_winner():
        board = board_set.winning_board()
        break
print(">> Winning board:")
print(board)
print(">> Marks:")
print(board.marks)
print(f">> Score: {board.score}")

>> Winning board:
70 56 80 12 11
35 55 40 71 87
84 27 96 46 85
20 23 26 29 14
58 37 21 75 68
>> Marks:
[85, 84, 30, 15, 46, 71, 64, 45, 13, 90, 63, 89, 62, 25, 87, 68, 73, 47, 65, 78, 2, 27, 67, 95, 88, 99, 96]
>> Score: 63552


As we see above, we arrived at the final answer for this gold star, `62552`. And I'm happy to report that was the correct answer on the first attempt of this code! 😊

## Part 2

This appears slightly more difficult at first, but actually I can completely re-use everything previously written without changes. Though I will have to muck about in the internals ever so slightly.

To locate the last winning board, all I need to do is exclude winners from the board set until only one board remains. When that last board is left standing, then we just keep drawing until it wins and grab its score. Simple enough.

In [38]:
# I'll reset all board inputs to ensure we have a clean start
board_set = BoardSet()
board_set.boards.extend(
    [BingoBoard(board=x) for x in bingo_board_inputs()]
)

# and restart the draw
for num in numbers_drawn():
    board_set.mark(num)
    if board_set.has_winner():
        if len(board_set.boards) > 1:
            # exclude earlier winners from the board set
            board_set.boards = [x for x in board_set.boards if not x.is_winner()]
        else:
            # we found the final winner!
            final_board = board_set.boards[0]
            break

print(">> Last winning board:")
print(final_board)
print(">> Marks:")
print(final_board.marks)
print(f">> Score: {final_board.score}")

>> Last winning board:
50 66 43 39 16
88 94 60 70 64
63 80 56 69 36
53 48 32 22 79
59 77 20 30 67
>> Marks:
[85, 84, 30, 15, 46, 71, 64, 45, 13, 90, 63, 89, 62, 25, 87, 68, 73, 47, 65, 78, 2, 27, 67, 95, 88, 99, 96, 17, 42, 31, 91, 98, 57, 28, 38, 93, 43, 0, 55, 49, 22, 24, 82, 54, 59, 52, 3, 26, 9, 32, 4, 48, 39, 50, 80, 21, 5, 1, 23, 10, 58, 34, 12, 35, 74, 8, 6, 79, 40, 76, 86, 69, 81, 61, 14, 92, 97, 19, 7, 51, 33, 11, 77, 75, 20]
>> Score: 9020


And again, first run answer was correct! 🎉